In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import pandas as pd
import autograd.numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image

from utils.models import BNN, BNN_LV
from utils.functions import gaussian, log_gaussian
from utils.training import HMC
from utils.data_gen import sample_gaussian_mixture, generate_regression_outputs


In [ ]:
# Data Loading
y_bimodal, X_bimodal = generate_regression_outputs(type='bimodal')


In [ ]:
# PDFs
# Set up the prior, likelihood and posterior
def log_prior(W, mu, sigma):
    """ Generate the prior PDF """
    return np.sum(log_gaussian(x=W, mu=mu, sigma=sigma), axis=-1)

# Set up the prior, likelihood and posterior
def log_latent_prior(z, mu, gamma):
    """ Generate the prior PDF """
    return np.sum(np.sum(log_gaussian(x=z, mu=mu, sigma=gamma), axis=-1), axis=-1)

def log_likelihood(W, X, Y, mu, sigma):
    """ Generate the likelihood PDF """
    llh = np.sum(log_gaussian(x=Y, mu=mu, sigma=sigma), axis=0)
    return llh

def create_log_posterior(X, Y, p_mu, p_sigma, l_sigma, nn, gamma):
    """ Wrapper to create an initialized posterior PDF """
    def log_posterior(W, X=X, Y=Y, p_mu=p_mu, p_sigma=p_sigma, l_sigma=l_sigma, nn=nn, gamma=gamma):
        """ Generate the posterior PDF """
        mu_l = nn.forward(X, weights=W)
        z = nn.last_input_noise
        log_p = log_prior(W=W, mu=p_mu, sigma=p_sigma)
        log_zp = log_latent_prior(z=z, mu=0, gamma=gamma)
        log_l = log_likelihood(W=W, X=X, Y=Y, mu=mu_l, sigma=l_sigma)
        llh = log_p + log_l + log_zp
        return llh 
    return log_posterior
    

In [ ]:
# Parameters
gamma = 1
sigma = 1

architecture = {'input_n':1, 
             'output_n':1, 
             'hidden_layers':[5],
             'biases' : [1,1],
             'activations' : ['relu', 'linear'],
             'gamma':[gamma],
             'sigma':[sigma]}
bnn_lv = BNN_LV(architecture=architecture)

bnn_lv.fit(X_bimodal, y_bimodal, step_size=0.01, max_iteration=5000, check_point=500, regularization_coef=None)


Iteration 0 lower bound 63.260152998578754; gradient mag: 24.27741522524019
Iteration 500 lower bound 34.4354849622109; gradient mag: 0.03338015240680576
Iteration 1000 lower bound 34.434765228208065; gradient mag: 0.0031400830220036135
Iteration 1500 lower bound 34.434765850964915; gradient mag: 0.006095059901687306
Iteration 2000 lower bound 34.43476661444538; gradient mag: 0.0061293490755887905
Iteration 2500 lower bound 34.43476877342321; gradient mag: 0.012461237542253185
Iteration 3000 lower bound 34.434771856582735; gradient mag: 0.043076357842016515
Iteration 3500 lower bound 34.43477186065011; gradient mag: 0.011282303947488585
Iteration 4000 lower bound 34.434770328611094; gradient mag: 0.014129108523134103
Iteration 4500 lower bound 34.43477448589385; gradient mag: 0.044563473488649266


In [ ]:
# Create the posterior
log_posterior_bnn_lv = create_log_posterior(X_bimodal, y_bimodal, 0, 5, 0.25, bnn_lv, gamma=gamma)

# Run HMC from randiom intial starting weights
mle_weights = bnn_lv.get_weights()

hmc = HMC(log_target_func=log_posterior_bnn_lv, position_init=mle_weights, total_samples=1000, leapfrog_steps=1000, step_size=2e-4,
    burn_in=0.5, thinning_factor=1, mass=1.0, random_seed=None, progress=False)

# Train the Model
bayesian_lv_weights = hmc.sample()

# Load a pretrained model
# hmc.load_state(filepath='../data/hmc/heteroscedastic_run1.json')
# bayesian_lv_weights = hmc.samples

/opt/venv/lib/python3.7/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)


KeyboardInterrupt: 

In [ ]:
# Set up data
x_test_q3 = np.linspace(-6, 6, 100)

# Take 100 random posterior samples
w_random_samples = bayesian_lv_weights[np.random.choice(bayesian_lv_weights.shape[0], 10000), :]

y_preds = []

# Loop through the samples of weights
for i in range(w_random_samples.shape[0]):
    # Create the same NN for predictions but with weights from the samples
    w_cur = w_random_samples[i,:]

    mu_pred = bnn_lv.forward(x_test_q3.reshape(-1,1), w_cur)
    y_pred = mu_pred
    y_preds.append(y_pred.reshape(-1))

# Calculate percentiles
y_lower = np.percentile(y_preds, q=2.5, axis=0)
y_upper = np.percentile(y_preds, q=97.5, axis=0)
y_med = np.percentile(y_preds, q=50, axis=0)

# Plot with confidence
plt.figure(figsize=(14,7))
plt.scatter(X_hsc.flatten(), y_hsc.flatten(), color='black', label='data')
plt.plot(x_test_q3, y_med, label="Median Prediction")
plt.fill_between(x_test_q3, y_lower, y_upper, alpha=0.4, color='r', label="95% Predictive Interval")
plt.title("Bayesian Neural Net Predictions with 95% CI")
plt.xlabel("X Test")
plt.ylabel("Y Predicted")
plt.legend()
plt.show()


In [ ]:
# Save the weights from HMC:
hmc.save_state(filepath='../data/hmc/bimodal_run1.json', replace=True)